1. Import Required Libraries

In [1]:
import os
import random

# Import PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
import torchvision.models as models
from torchvision.models import EfficientNet_B0_Weights
from torchvision.transforms import ToTensor, Resize, Normalize
from torchvision.transforms import InterpolationMode

# Import matplotlib for visualization
import matplotlib
import matplotlib.pyplot as plt

from PIL import Image

from tqdm.auto import tqdm

from datetime import datetime
from timeit import default_timer as timer

from torch.cuda.amp import GradScaler, autocast  # Import GradScaler and autocast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Check versions
print(torch.__version__)
print(torchvision.__version__)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

2.5.1+cu124
0.20.1+cu124
1
NVIDIA GeForce RTX 3050 6GB Laptop GPU


2. Set the Hyperparameters

In [2]:
# Paths to real and fake image directories
train_real_path = r"D:\Projects\SP CUP Dataset\train\real" # r is because of the backslash causing \p 
train_fake_path = r"D:\Projects\SP CUP Dataset\train\fake" # to be interpreted as a special character
train_path = r"D:\Projects\SP CUP Dataset\train"

test_real_path = r"D:\Projects\SP CUP Dataset\valid\real"
test_fake_path = r"D:\Projects\SP CUP Dataset\valid\fake"
test_path = r"D:\Projects\SP CUP Dataset\valid"

#Small Dataset
#train_real_path = r"D:\Projects\SP CUP Dataset\valid\real" # r is because of the backslash causing \p 
#train_fake_path = r"D:\Projects\SP CUP Dataset\valid\fake" # to be interpreted as a special character
#test_real_path = r"D:\Projects\SP CUP Dataset\Small train\real"
#test_fake_path = r"D:\Projects\SP CUP Dataset\Small train\fake"

#Define the maximum number of samples per class
TRAIN_MAX_SAMPLES_PER_CLASS = 50000  # Programmer-defined limit
TEST_MAX_SAMPLES_PER_CLASS = 50000  # Programmer-defined limit

BATCH_SIZE = 16 # Batch size for training
#LR = 0.1 # Learning rate
RANDOM_SEED = 42 # Seed for random number generator
Visualize = False # Visualize the data

#Define the classes
classes = ['Fake', 'Real']
Real_Index = 1
Fake_Index = 0

num_epochs = 10

✔ 1 for real and 0 for fake since classes = ['fake', 'real']
fileID <TAB> score where fileID is the id of the test file, and score is a numerical value -- a higher value for real images and lower value for fake images.(from documentation)

✔ The train_dataloader is created with shuffle=True to shuffle the training data at the beginning of each epoch. (not implemented )

✔ The test_dataloader is created with shuffle=False to keep the order of the test data consistent.

✔Read efficientnet documentation and act accordingly

Way to save the model using checkpoints.

Way to get std and mean of the provided dataset

Apply regularization (Model markdown)

3. Balanced Training and Testing Datasets

In [3]:
def display_total_filecount(real_path, fake_path):
    print(f"Number of real images: {len(os.listdir(real_path))}, "
          f"Number of fake images: {len(os.listdir(fake_path))}")

In [4]:
display_total_filecount(train_real_path, train_fake_path)
display_total_filecount(test_real_path, test_fake_path)

Number of real images: 42690, Number of fake images: 219470
Number of real images: 1548, Number of fake images: 1524


3.1 Create Balanced Datasets

In [5]:
#random.seed(RANDOM_SEED)

#Get the image filenames
train_real_images = os.listdir(train_real_path)
train_fake_images = os.listdir(train_fake_path)

# Apply the limit
train_real_images = train_real_images[:min(TRAIN_MAX_SAMPLES_PER_CLASS, len(train_real_images))]
#train_fake_images = train_fake_images[:min(TRAIN_MAX_SAMPLES_PER_CLASS, len(train_fake_images))]
#Added Choosing random to avoid training only for a sample of the 200000 images
train_fake_images = random.sample(train_fake_images, min(TRAIN_MAX_SAMPLES_PER_CLASS, len(train_fake_images)))

# Oversample real_images to match the number of fake_images
if len(train_real_images) < len(train_fake_images):
    train_real_images = random.choices(train_real_images, k=len(train_fake_images))
    
# Combine and label the dataset
train_balanced_dataset = [(os.path.join(train_real_path, img), Real_Index) for img in train_real_images] + \
                   [(os.path.join(train_fake_path, img), Fake_Index) for img in train_fake_images]

# Shuffle the dataset
random.shuffle(train_balanced_dataset)

# Verify the limited dataset size
print(f"Total dataset size: {len(train_balanced_dataset)} (Real: {len(train_real_images)}, Fake: {len(train_fake_images)})")


Total dataset size: 100000 (Real: 50000, Fake: 50000)


In [7]:
import os
import random

def prepare_balanced_dataset(train_real_path, train_fake_path, real_index, fake_index, max_samples_per_class):
    """
    Prepare a balanced dataset with oversampled real images and randomly sampled fake images.

    Args:
        train_real_path (str): Path to the real images.
        train_fake_path (str): Path to the fake images.
        real_index (int): Label/index for real images.
        fake_index (int): Label/index for fake images.
        max_samples_per_class (int): Maximum number of samples per class for training.

    Returns:
        list: A balanced dataset as a list of tuples (image_path, label).
    """
    # Get the image filenames
    train_real_images = os.listdir(train_real_path)
    train_fake_images = os.listdir(train_fake_path)

    # Apply the limit to real and fake images
    train_real_images = train_real_images[:min(max_samples_per_class, len(train_real_images))]
    train_fake_images = random.sample(train_fake_images, min(max_samples_per_class, len(train_fake_images)))

    # Oversample real images to match the number of fake images
    if len(train_real_images) < len(train_fake_images):
        train_real_images = random.choices(train_real_images, k=len(train_fake_images))

    # Combine and label the dataset
    train_balanced_dataset = [
        (os.path.join(train_real_path, img), real_index) for img in train_real_images
    ] + [
        (os.path.join(train_fake_path, img), fake_index) for img in train_fake_images
    ]

    # Shuffle the dataset
    random.shuffle(train_balanced_dataset)

    # Print the dataset size for verification
    print(f"Total dataset size: {len(train_balanced_dataset)} (Real: {len(train_real_images)}, Fake: {len(train_fake_images)})")

    #Create the dataset
    train_data = ImagDataset(train_balanced_dataset,
                              transform=train_transform)

    return train_balanced_dataset



Total dataset size: 100000 (Real: 50000, Fake: 50000)


In [6]:
# Get the image filenames
test_real_images = os.listdir(test_real_path)
test_fake_images = os.listdir(test_fake_path)

# Apply the limit
test_real_images = test_real_images[:min(TEST_MAX_SAMPLES_PER_CLASS, len(test_real_images))]
test_fake_images = test_fake_images[:min(TEST_MAX_SAMPLES_PER_CLASS, len(test_fake_images))]

# Combine and label the dataset
test_balanced_dataset = [(os.path.join(test_real_path, img), Real_Index) for img in test_real_images] + \
                   [(os.path.join(test_fake_path, img), Fake_Index) for img in test_fake_images]

# Shuffle the dataset
random.shuffle(test_balanced_dataset)

# Verify the limited dataset size
print(f"Total dataset size: {len(test_balanced_dataset)} (Real: {len(test_real_images)}, Fake: {len(test_fake_images)})")

Total dataset size: 3072 (Real: 1548, Fake: 1524)


3.2 Visualize balanced datasets

In [7]:
def display_random_images(dataset, num_images):
    # Set the random seed for reproducibility
    random.seed(RANDOM_SEED)
    
    # Randomly select num_images from the dataset
    selected_indices = random.sample(range(len(dataset)), num_images)
    
    # Create a figure with subplots
    fig, axes = plt.subplots(1, num_images, figsize=(20, 8))  # Adjust the figsize as needed
    
    for i, idx in enumerate(selected_indices):
        image_path, label = dataset[idx]
        image = Image.open(image_path)
        
        # Display the image
        axes[i].imshow(image)
        axes[i].set_title(f"Label: {'Real' if label == Real_Index else 'Fake'}")
        axes[i].set_xlabel(os.path.basename(image_path))  # Display the image name
        axes[i].axis('off')  # Hide the axis
        
        # Print the image file name and shape
        print(f"Image file name: {os.path.basename(image_path)}")
        print(f"Image Shape: {image.size}")
    
    plt.tight_layout()
    plt.show()

# Example usage with train_balanced_dataset and a seed value
if Visualize==True:
    display_random_images(train_balanced_dataset, num_images=5)

4. Transformed Datasets

4.1 Image Dataset Class

In [8]:
# Define the dataset class
class ImageDataset(Dataset):
    def __init__(self, data, transform=None):
        super().__init__()  # Call the parent class's init method
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

4.2 Finding out mean and std to use in the transform

In [9]:
def compute_mean_and_std_from_dataset(dataset, device='cuda'):
    """
    Compute per-channel mean and std of the dataset (to be used in transforms.Normalize())
    Args:
        dataset (list): List of tuples containing image paths and labels.
        device (str): Device to perform computations on ('cuda' or 'cpu').
    Returns:
        mean (list): List of mean values for each channel.
        std (list): List of standard deviation values for each channel.
    """
    # Initialize variables to store the sum and sum of squares of pixel values
    mean = torch.zeros(3).to(device)
    std = torch.zeros(3).to(device)
    num_images = len(dataset)
    
    for img_path, _ in tqdm(dataset, desc="Computing mean and std"):
        image = Image.open(img_path).convert("RGB")
        image = transforms.ToTensor()(image).to(device)
        
        mean += image.mean(dim=(1, 2))
        std += image.std(dim=(1, 2))
    
    mean /= num_images
    std /= num_images

    mean = [round(m.item(), 4) for m in mean]
    std = [round(s.item(), 4) for s in std]
    
    return mean, std

In [10]:

# Example usage
#train_mean, train_std = compute_mean_and_std_from_dataset(train_balanced_dataset)
#print(f"Train Mean: {train_mean}, Train Std: {train_std}")

In [11]:
#test_mean, test_std = compute_mean_and_std_from_dataset(test_balanced_dataset)
#print(f"Test Mean: {test_mean}, Test Std: {test_std}")

4.3 Create transformed datasets

In [ ]:
train_mean, train_std = [0.4598, 0.3929, 0.3792], [0.2327, 0.2046, 0.2103]
test_mean, test_std = [0.3996, 0.3194, 0.3223], [0.2272, 0.1706, 0.1718]

#train_mean, train_std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
#test_mean, test_std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

#This transform has better accuracy than the one below
# Define transforms
#transform = transforms.Compose([
#    Resize((224, 224)),
#    ToTensor(),
#    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#])

train_transform = transforms.Compose([
    transforms.Resize(224, interpolation=InterpolationMode.BICUBIC),  # First resize to 256
    #transforms.CenterCrop(224),  # Then center crop to 224
    transforms.RandomHorizontalFlip(p=0.5),  # Flip image horizontally with 50% probability
    transforms.ToTensor(),
    #transforms.Normalize(mean=train_mean.tolist(), std=train_std.tolist())

    #transforms.ColorJitter(
    #                brightness=0.2,      # Random brightness adjustment
    #                contrast=0.2,        # Random contrast adjustment
    #                saturation=0.2,      # Random saturation adjustment
    #                hue=0.1             # Random hue adjustment
    #            ),

    transforms.Normalize(mean=train_mean, std=train_std)

])

test_transform = transforms.Compose([
    transforms.Resize(224, interpolation=InterpolationMode.BICUBIC),  # First resize to 256
    #transforms.CenterCrop(224),  # Then center crop to 224
    transforms.ToTensor(),
    #transforms.Normalize(mean=test_mean.tolist(), std=test_std.tolist())
    transforms.Normalize(mean=test_mean, std=test_std)
])



#Create the dataset
test_data = ImageDataset(test_balanced_dataset, 
                        transform=test_transform)

# Example: Print the class name of a label
for image, label in test_data:
    class_name = classes[label]
    print(f"Image label: {label}, Class name: {class_name}")
    break  # Just print the first one for demonstration


Image label: 0, Class name: Fake


4.4 Visualize transformed datasets

In [13]:
def unnormalize(tensor, mean, std):
    """
    Unnormalize a tensor image.
    Args:
        tensor (torch.Tensor): The normalized tensor image.
        mean (torch.Tensor): The mean used for normalization.
        std (torch.Tensor): The standard deviation used for normalization.
    Returns:
        torch.Tensor: The unnormalized tensor image.
    """
    tensor = tensor.clone()  # Clone the tensor to avoid modifying the original
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)  # Unnormalize
    return tensor

def plot_random_images(dataset, transform, num_images):
    random.seed(RANDOM_SEED)
    
    # Randomly select num_images from the dataset
    selected_indices = random.sample(range(len(dataset)), num_images)
    
    # Display num_images before and after transform
    fig, axes = plt.subplots(3, num_images, figsize=(20, 8))  # 2 rows, num_images columns
    
    for i, idx in enumerate(selected_indices):
        image_path, label = dataset[idx]
        image = Image.open(image_path).convert("RGB")
        
        # Display original image
        axes[0, i].imshow(image)
        axes[0, i].set_title(f"Original\nLabel: {'Real' if label == Real_Index else 'Fake'}")
        axes[0, i].axis('off')  # Hide the axis

        # Apply transform and display transformed image
        transformed_image = transform(image)
        axes[1, i].imshow(transformed_image.permute(1, 2, 0))  # Display the transformed image
        axes[1, i].set_title(f"Transformed\nLabel: {'Real' if label == Real_Index else 'Fake'}")
        axes[1, i].axis('off')  # Hide the axis

        # Unnormalize and display the unnormalized image
        unnormalized_image = unnormalize(transformed_image, train_mean, train_std)
        axes[2, i].imshow(unnormalized_image.permute(1, 2, 0))  # Unnormalize the image
        axes[2, i].set_title(f"Unnormalized\nLabel: {'Real' if label == Real_Index else 'Fake'}")
        axes[2, i].axis('off')  # Hide the axis

        # Print the transformed image shape and label
        print(transformed_image.shape, label)
        print(f"Image file name: {image_path}")
        print(f"Image label: {[label]}")


    plt.tight_layout()
    plt.show()

# Example usage with train_balanced_dataset and train_transform
if Visualize==True:
    plot_random_images(train_balanced_dataset, train_transform, num_images=5)


5. DataLoaders

5.1 Create DataLoaders

In [14]:
# Create the DataLoader
train_dataloader = DataLoader(train_data, 
                        batch_size=BATCH_SIZE,
                        shuffle=True)

test_dataloader = DataLoader(test_data, 
                        batch_size=BATCH_SIZE,
                        shuffle=False)


5.2 Visualize DataLoaders

In [15]:
# Example: Iterate through the DataLoader
for images, labels in test_dataloader:
    print(images.shape, labels.shape)
    break

print(images[0].shape)


# Let's check out what what we've created
print(f"DataLoaders: {train_dataloader, test_dataloader}")
print(f"Length of train_dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}... = {BATCH_SIZE*len(train_dataloader)}")
print(f"Length of test_dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}... = {BATCH_SIZE*len(test_dataloader)}")

torch.Size([16, 3, 224, 224]) torch.Size([16])
torch.Size([3, 224, 224])
DataLoaders: (<torch.utils.data.dataloader.DataLoader object at 0x000001AF51D22300>, <torch.utils.data.dataloader.DataLoader object at 0x000001AF1B33B5C0>)
Length of train_dataloader: 6250 batches of 16... = 100000
Length of test_dataloader: 192 batches of 16... = 3072


6. Building a Model

6.0 Model_v0

In [16]:
# Load pre-trained EfficientNet model
class DeepFakeDetectV0(nn.Module):
    def __init__(self, output_shape: int):
        super(DeepFakeDetectV0, self).__init__()
        # Load pre-trained EfficientNet model
        self.efficientnet = models.efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        
        # Unfreeze more layers
        for param in self.efficientnet.parameters():
            param.requires_grad = True
        
        # Replace the final fully connected layer
        num_ftrs = self.efficientnet.classifier[1].in_features
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(num_ftrs, output_shape)
        )
        
    def forward(self, x):
        return self.efficientnet(x)

7. Setup the model for the first time

In [17]:

model_0 = DeepFakeDetectV0(output_shape=1)
model_0.to(device)

# Check the device of the model
model_0_device = next(model_0.parameters()).device
print(f"Model is on device: {model_0_device}")
#model_0

Model is on device: cuda:0


7.1 Visualize Model's Output

In [18]:
if Visualize == True:
    # Test with random tensor (batch_size=32, channels=3, height=224, width=224)
    test_input = torch.randn(32, 3, 224, 224).to(device)
    output = model_0(test_input)
    print(f"Input shape: {test_input.shape}")
    print(f"Output shape: {output.shape}")
    print(f"Sample predictions:\n{output[:5].cpu().detach().numpy()}")

7.2 Check the State Dictionary of the model

In [19]:
if Visualize == True:
    print(model_0.state_dict())

7.3 Further Checking the Model

In [20]:
if Visualize == True:
    # Iterate through the test_dataloader and print the first 5 values
    for i, (images, labels) in enumerate(test_dataloader):
        if i < 1:
            print(f"Batch {i+1}:")
            print(f"Images: {images}")
            print(f"Labels: {labels}")
        else:
            break

In [21]:
if Visualize == True:
    # View the first 5 outputs of the forward pass on the test data
    model_0.eval()
    with torch.inference_mode():
      for X_test, y_test in test_dataloader:
        X_test, y_test = X_test.to(device), y_test.to(device)  # Move tensors to the same device as the model
        # 1. Forward pass
        y_logits = model_0(X_test.to(device))[:5]
        break
      
    print("y_logits ",y_logits)
    print("y_test ", y_test[:5])

    # Use the sigmoid activation function on our model logits to turn them into prediction probabilities
    y_pred_probs = torch.sigmoid(y_logits)
    print("y_pred_probs ",y_pred_probs)
    # Find the predicted labels
    y_preds = torch.round(y_pred_probs)

    # Convert the tensor to a list of labels
    y_pred_labels = [classes[int(label)] for label in y_preds]

    print(y_pred_labels)

8. Setup Loss, Optimizer and evaluation metrics

8.1 Setup Loss and Optimizer

In [22]:
# Setup loss function and optimizer
#loss_fn = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.SGD(params=model_0.parameters(),
                            #lr=0.01)
# Define the optimizer
#optimizer = optim.Adam(model_0.parameters(), lr=LR)

# Define the learning rate scheduler
#scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

# Setup loss function and optimizer
loss_fn = nn.BCEWithLogitsLoss()

#Define optimizer with initial learning rate
initial_lr = 0.003  # EfficientNet often works well with lower learning rates
optimizer = optim.Adam(model_0.parameters(), lr=initial_lr)

# Create scheduler with OneCycleLR - works well for deepfake detection
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=initial_lr,  
    epochs=num_epochs,
    steps_per_epoch=len(train_dataloader),
    pct_start=0.3,      # Warm up for 30% of training
    anneal_strategy='cos',
    div_factor=10,      # initial_lr/10 will be used as starting lr
    final_div_factor=100  # final lr will be initial_lr/1000
)

8.2 Function to time our experiments

In [23]:
from timeit import default_timer as timer
def print_train_time(start: float,
                     end: float,
                     device: torch.device = None):
  """Prints difference between start and end time."""
  total_time = end - start
  print(f"Train time on {device}: {total_time:.3f} seconds")
  return total_time

start_time = timer()
# some code...
end_time = timer()
print_train_time(start=start_time, end=end_time, device="cpu")

Train time on cpu: 0.000 seconds


1.3900000340072438e-05

8.3 Accuracy Function

In [24]:
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred)) * 100
  return acc

In [25]:
# Initialize GradScaler for mixed precision training
scaler = torch.cuda.amp.GradScaler(enabled=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_12108\4065757075.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


In [26]:
from datetime import datetime
model_save_path = f"deepfake_detect_model_20250112_202517.pth"

# Load the trained model
loaded_model = DeepFakeDetectV0(output_shape=1)
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.to(device)
print("Model loaded successfully")

model_0 = loaded_model

Model loaded successfully


C:\Users\ASUS\AppData\Local\Temp\ipykernel_12108\1629135735.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(model_save_path))


9. Creating a Training loop and training the model

In [ ]:
# Set the seed and start the timer
torch.manual_seed(42)
torch.cuda.manual_seed(42)
random.seed(42)
train_time_start_on_cpu = timer()

# Set the number of epochs (we'll keep this small for faster training time)
epochs = num_epochs

# Create training and test loop
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n------")

  balanced_dataset = prepare_balanced_dataset(
    train_real_path, 
    train_fake_path, 
    Real_Index, 
    Fake_Index, 
    TRAIN_MAX_SAMPLES_PER_CLASS)
  
  ### Training
  train_loss, train_acc = 0, 0
  batch_accuracy = 0

  # Add a loop to loop through the training batches
  for batch, (X, y) in enumerate(tqdm(train_dataloader)):
    # Put data to target device
    X, y = X.to(device), y.float().to(device)
    
    model_0.train()
    
    optimizer.zero_grad()

    with torch.cuda.amp.autocast():
        # 1. Forward pass
        y_pred = model_0(X).squeeze()
        #print(y_pred.type)
        #print(y_pred[0])

        # 2. Calculate loss (per batch)
        loss = loss_fn(y_pred.view(-1), y.float())

    
    train_loss += loss.item() # accumulate train loss

    # 3. Optimizer zero grad
    #optimizer.zero_grad()
    
    # 4. Loss backward
    #loss.backward()
    scaler.scale(loss).backward()

    # 5. Optimizer step (update the model's parameters once *per batch*)
    optimizer.step()
    #scaler.step(optimizer)
    #scaler.update()

    with torch.no_grad():
        # Calculate accuracy
        y_pred_class = torch.round(torch.sigmoid(y_pred))
        #print(y_pred_class)
        #print(y_pred_class.view(-1))
        batch_accuracy = (y_pred_class.view(-1) == y).sum().item() / len(y_pred_class)
        #print(f"Batch accuracy: {batch_accuracy}")

        # Accumulate the batch accuracy to the training accuracy
        train_acc += batch_accuracy
        #print(f"Cumulative training accuracy: {train_acc}")

    # Print out what's happening
    if batch % 400 == 0:
        print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples.")

    # Add at the end of each epoch in the training loop
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch} ending learning rate: {current_lr:.6f}")
    scheduler.step()  # Update learning rate at each step

  # Divide total train loss by length of train dataloader
  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  print(f"\nTrain loss: {train_loss:.4f}")
  print(f"Cumulative training accuracy: {train_acc}")

  ### Testing
  test_loss, correct_predictions, total_predictions = 0, 0, 0
  real_as_real, real_as_fake, fake_as_real, fake_as_fake = 0, 0, 0, 0
  model_0.eval()
  with torch.inference_mode():
      for i, (X_test, y_test) in enumerate(test_dataloader):
          # Put data to target device
          X_test, y_test = X_test.to(device), y_test.float().to(device)

          # 1. Forward pass
          test_pred = model_0(X_test).squeeze()

          # 2. Calculate loss (accumulatively)
          test_loss += loss_fn(test_pred, y_test.float()).item()

          # 3. Calculate accuracy
          test_pred_class = torch.round(torch.sigmoid(test_pred))
          correct_predictions += (test_pred_class.view(-1) == y_test).sum().item()
          total_predictions += len(y_test)

          # Print image names and predictions
          for j in range(len(X_test)):
              image_index = i * test_dataloader.batch_size + j
              image_name = test_dataloader.dataset.data[image_index][0]
              true_label = test_dataloader.dataset.data[image_index][1]
              predicted_label = test_pred_class[j].item()
              #print(f"Image: {image_name}, True Label: {true_label}, Predicted Label: {predicted_label}")
              #print(f"Image: {image_name}, True Label: {classes[int(true_label)]}, Predicted Label: {classes[int(predicted_label)]}")

              if true_label == 0 and predicted_label == 0:
                  real_as_real += 1
              elif true_label == 0 and predicted_label == 1:
                  real_as_fake += 1
              elif true_label == 1 and predicted_label == 0:
                  fake_as_real += 1
              elif true_label == 1 and predicted_label == 1:
                  fake_as_fake += 1
              #print(real_as_real, real_as_fake, fake_as_real, fake_as_fake)

      # Calculate the test loss average per batch
      test_loss /= len(test_dataloader)

      # Calculate the test accuracy
      #test_acc = correct_predictions / total_predictions
      test_acc = (real_as_real + fake_as_fake) / (real_as_real + real_as_fake + fake_as_real + fake_as_fake)
      print(f"Real images identified as real: {real_as_real}")
      print(f"Real images identified as fake: {real_as_fake}")
      print(f"Fake images identified as real: {fake_as_real}")
      print(f"Fake images identified as fake: {fake_as_fake}")
      print(f"\nReal images Total: {real_as_real + real_as_fake} | Fake images Total: {fake_as_real + fake_as_fake}")
      print(f"\nTest loss: {test_loss:.4f}, Test acc: {test_acc:.4f}")


  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
  model_save_path = f"deepfake_detect_model_{timestamp}.pth"
  torch.save(model_0.state_dict(), model_save_path)
  print(f"Model saved to {model_save_path}")

  # Print out what's happening
  print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc*100:.4f}% | Test loss: {test_loss:.4f} | Test acc: {test_acc*100:.4f}%")


# Calculate training time
train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu,
                                            end=train_time_end_on_cpu,
                                            device=str(next(model_0.parameters()).device))

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
------


  0%|          | 0/6250 [00:00<?, ?it/s]

C:\Users\ASUS\AppData\Local\Temp\ipykernel_12108\3491243248.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Looked at 0/100000 samples.
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.000300
Epoch 0 ending learning rate: 0.0003

...

  0%|          | 0/6250 [00:00<?, ?it/s]

Looked at 0/100000 samples.
Epoch 2 ending learning rate: 0.002325
Epoch 2 ending learning rate: 0.002325
Epoch 2 ending learning rate: 0.002326
Epoch 2 ending learning rate: 0.002326
Epoch 2 ending learning rate: 0.002326
Epoch 2 ending learning rate: 0.002326
Epoch 2 ending learning rate: 0.002326
Epoch 2 ending learning rate: 0.002327
Epoch 2 ending learning rate: 0.002327
Epoch 2 ending learning rate: 0.002327
Epoch 2 ending learning rate: 0.002327
Epoch 2 ending learning rate: 0.002327
Epoch 2 ending learning rate: 0.002327
Epoch 2 ending learning rate: 0.002328
Epoch 2 ending learning rate: 0.002328
Epoch 2 ending learning rate: 0.002328
Epoch 2 ending learning rate: 0.002328
Epoch 2 ending learning rate: 0.002328
Epoch 2 ending learning rate: 0.002329
Epoch 2 ending learning rate: 0.002329
Epoch 2 ending learning rate: 0.002329
Epoch 2 ending learning rate: 0.002329
Epoch 2 ending learning rate: 0.002329
Epoch 2 ending learning rate: 0.002330
Epoch 2 ending learning rate: 0.0023

  0%|          | 0/6250 [00:00<?, ?it/s]

Looked at 0/100000 samples.
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.003000
Epoch 3 ending learning rate: 0.0030

KeyboardInterrupt: 

In [ ]:
#from datetime import datetime
#model_save_path = f"deepfake_detect_model_20250110_201404.pth"

# Load the trained model
#loaded_model = DeepFakeDetectV0(output_shape=1)
#loaded_model.load_state_dict(torch.load(model_save_path))
#loaded_model.to(device)
#print("Model loaded successfully")



In [ ]:
from datetime import datetime
model_save_path = f"deepfake_detect_model_20250112_122939.pth"

# Load the trained model
loaded_model = DeepFakeDetectV0(output_shape=1)
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.to(device)
print("Model loaded successfully")

model_0 = loaded_model

In [27]:
### Testing
test_loss, correct_predictions, total_predictions = 0, 0, 0
real_as_real, real_as_fake, fake_as_real, fake_as_fake = 0, 0, 0, 0
model_0.eval()
with torch.inference_mode():
    for i, (X_test, y_test) in enumerate(test_dataloader):
        # Put data to target device
        X_test, y_test = X_test.to(device), y_test.float().to(device)

        # 1. Forward pass
        test_pred = model_0(X_test).squeeze()

        # 2. Calculate loss (accumulatively)
        test_loss += loss_fn(test_pred, y_test.float()).item()

        # 3. Calculate accuracy
        test_pred_class = torch.round(torch.sigmoid(test_pred))
        correct_predictions += (test_pred_class.view(-1) == y_test).sum().item()
        total_predictions += len(y_test)

        # Print image names and predictions
        for j in range(len(X_test)):
            image_index = i * test_dataloader.batch_size + j
            image_name = test_dataloader.dataset.data[image_index][0]
            true_label = test_dataloader.dataset.data[image_index][1]
            predicted_label = test_pred_class[j].item()
            #print(f"Image: {image_name}, True Label: {true_label}, Predicted Label: {predicted_label}")
            #print(f"Image: {image_name}, True Label: {classes[int(true_label)]}, Predicted Label: {classes[int(predicted_label)]}")

            if true_label == 1 and predicted_label == 1:
                real_as_real += 1
            elif true_label == 1 and predicted_label == 0:
                real_as_fake += 1
            elif true_label == 0 and predicted_label == 1:
                fake_as_real += 1
            elif true_label == 0 and predicted_label == 0:
                fake_as_fake += 1
            #print(real_as_real, real_as_fake, fake_as_real, fake_as_fake)

    # Calculate the test loss average per batch
    test_loss /= len(test_dataloader)

    # Calculate the test accuracy
    #test_acc = correct_predictions / total_predictions
    test_acc = (real_as_real + fake_as_fake) / (real_as_real + real_as_fake + fake_as_real + fake_as_fake)
    print(f"Real images identified as real: {real_as_real}")
    print(f"Real images identified as fake: {real_as_fake}")
    print(f"Fake images identified as real: {fake_as_real}")
    print(f"Fake images identified as fake: {fake_as_fake}")
    print(f"\nReal images Total: {real_as_real + real_as_fake} | Fake images Total: {fake_as_real + fake_as_fake}")
    print(f"\nTest loss: {test_loss:.4f}, Test acc: {test_acc:.4f}")

# Print out what's happening
print(f"Test loss: {test_loss:.4f}, Test acc: {test_acc:.4f}")


Real images identified as real: 1277
Real images identified as fake: 271
Fake images identified as real: 487
Fake images identified as fake: 1037

Real images Total: 1548 | Fake images Total: 1524

Test loss: 0.5507, Test acc: 0.7533
Test loss: 0.5507, Test acc: 0.7533


In [ ]:
model_0.state_dict()